In [ ]:
from gurobipy import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.gridspec as gridspec # For weird grids (It is pretty cool)
print('Libraries imported')

##########################################################################################
##########################################################################################
##########################################################################################
# import balance data: 

import_balance = pd.read_csv('Balance.csv')
import_co2 = pd.read_csv('CO2_hourly.csv')
Data = pd.read_csv('Data_Baseline.csv')

time = Data['Unnamed: 0']
co2 = Data['CO2']
Balance = import_balance['Coupling']

##########################################################################################
##########################################################################################
##########################################################################################
# Create data dictionaries for the different variables
dict_hour = {}
dict_co2 = {}
dict_balance = {}
for i in time:
    dict_hour[time[i]] = time[i]
    dict_balance[time[i]] = Balance[i]
    dict_co2[time[i]] = co2[i]
##########################################################################################
##########################################################################################
##########################################################################################
#initiate model

m=Model()

print('Libraries read')
print('Data Ready')
print('Model initiated')
##########################################################################################
##########################################################################################
##########################################################################################
# Fixed variables

# Maximum battery change: 
B_input_max = 1

#Maximum charge in the battery
B_charge_max = 1.4

# Barttery efficiency
eff = np.sqrt(0.95)

##########################################################################################
##########################################################################################
##########################################################################################
# Decision variables: 
#define continuous variables
    #Electricity sold to the grid
sold = m.addVars(time, name="sold" , lb = 0, vtype= GRB.CONTINUOUS)
    #Electricity bought from the grid
bought = m.addVars(time, name="bought", lb = 0, vtype= GRB.CONTINUOUS)
    # Electricity from PV to battery
to_battery = m.addVars(time, name="to_battery", lb = 0, vtype= GRB.CONTINUOUS)
    #Electricity from battery to demand
from_battery = m.addVars(time, name="from_battery", lb = 0, vtype= GRB.CONTINUOUS)
    # Electricity charge of battery
Battery_state = m.addVars(time, name="Battery_state", lb = 0, ub = B_charge_max, vtype= GRB.CONTINUOUS)

##########################################################################################
##########################################################################################
##########################################################################################
# Define the objective: CO2 minimizing

objective = quicksum(bought[time[hour]] * dict_co2[time[hour]] 
                     for hour in time)

m.setObjective(objective, GRB.MINIMIZE)

##########################################################################################
##########################################################################################
##########################################################################################
# Define the constraints

# Positive balance value equals electricity sold and transferred to battery
m.addConstrs((dict_balance[hour] == (sold[hour] + to_battery[hour]) 
              for hour in time 
              if dict_balance[hour]> 0), name = 'Positive Balance');
# Define upper bound of electricity to the battery: 
m.addConstrs((to_battery[hour] <= dict_balance[hour] 
              for hour in time if dict_balance[hour] > 0), name = 'Positive Balance');

#Positive Balance : No electricity bought or taken from battery
m.addConstrs((bought[hour] == 0 
              for hour in time if dict_balance[hour] > 0), name = 'Positive Balance');
m.addConstrs((from_battery[hour] == 0 
              for hour in time if dict_balance[hour] > 0), name = 'Positive Balance');

# Negative balance value equals electricity bought and taken from battery
m.addConstrs((dict_balance[hour] == ((-1)*(bought[hour] + from_battery[hour])) 
              for hour in time
              if dict_balance[hour]<= 0), name = 'Negative Balance');
# Define maximum amount of battery that could be used at any point: 
m.addConstrs((from_battery[hour] <= (-1)*dict_balance[hour] 
              for hour in time if dict_balance[hour] <= 0), name = 'Negative Balance');

# Negative balance: no electricity sold or transferred to battery
m.addConstrs((sold[hour] == 0 
              for hour in time if dict_balance[hour] <= 0), name = 'Negative Balance');
m.addConstrs((to_battery[hour] == 0 
              for hour in time if dict_balance[hour] <= 0), name = 'Negative Balance');

# Initial battery state
m.addConstr((Battery_state[time[0]] == time[0]), name = 'Battery initial')

# battery state limits: 
m.addConstrs(( 0 <= Battery_state[time[hour]] for hour in time))
m.addConstrs(( Battery_state[time[hour]] <= B_charge_max for hour in time))

# Battery charged when balance is positive and discharged when balance is negative: 
# Added daily degradation
m.addConstrs((Battery_state[time[hour]] == Battery_state[time[hour-1]]*0.999999999 +to_battery[time[hour]]*eff 
              for hour in time
              if hour!= time[0]
              and dict_balance[hour] >= 0), name = 'Battery charging')

m.addConstrs((Battery_state[time[hour]] == Battery_state[time[hour-1]]*0.999999999 - from_battery[time[hour]]/eff 
              for hour in time
              if hour!= time[0]
              and dict_balance[hour]<= 0), name = 'Battery discharging')


# Maximum charge and discharge quantities: 
m.addConstrs((Battery_state[time[hour]] - Battery_state[time[hour-1]] <= B_input_max 
              for hour in time
              if hour!= time[0]
              and dict_balance[hour]> 0), name = 'Battery max charge per hour')

m.addConstrs((Battery_state[time[hour]] - Battery_state[time[hour-1]] >= (-1)*(B_input_max) 
              for hour in time
              if hour!= time[0]
              and dict_balance[hour]<= 0), name = 'Battery max discharge per hour')

# at t=0, no electricity can leave the battery (it somehow fails to do this)
m.addConstr((from_battery[0] == 0));

### Set time limit just in case: 
m.setParam('TimeLimit', 600);

##########################################################################################
##########################################################################################
##########################################################################################
# Solve
m.optimize()

##########################################################################################
##########################################################################################
##########################################################################################
# Variable extraction: 

# First I obtain all variables from the model. I use np.array because it otherwise returns a list. 
PV_Excess = np.array([sold[hour].X for hour in time])
Grid_Electricity = np.array([bought[hour].X for hour in time])
PV_to_battery = np.array([to_battery[hour].X for hour in time])
Electricity_from_battery = np.array([from_battery[hour].X for hour in time])
Battery_charge = np.array([Battery_state[hour].X for hour in time])

##########################################################################################
##########################################################################################
##########################################################################################
# Data for analysis:

Grid_PEF = 2.56
Solar_PEF = 1.0
PV_Consumption =  Data['PV [MWh]'] - PV_Excess - PV_to_battery

Optimization_data = pd.DataFrame({
    'Hour' : Data['Hour'],
    'Month' : Data['Month'],
    'Electricity price' : Data['Price'],
    'Demand [MWh]' : Data['Electricity [MWh]'],
    'Grid Electricity [MWh]' : Grid_Electricity,
    'PV Electricity [MWh]' : Data['PV [MWh]'],
    'PV Consumption [MWh]' : PV_Consumption,
    'PV Export [MWh]' : PV_Excess,
    'Battery-supplied Electricity [MWh]' : Electricity_from_battery,
    'Battery Electricity balance' : (-1)*PV_to_battery + Electricity_from_battery,
    'CO2 Emissions [kg]' : Grid_Electricity*Data['CO2'],
    'CO2 Avoided [kg]' : (PV_Consumption + Electricity_from_battery)*Data['CO2'],
    'CO2 Saved Exports [kg]' : PV_Excess * Data['CO2'],
    'Cost Electricity' : Grid_Electricity * Data['Price'] * 1000,
    'Cost saved PV' : (PV_Consumption + Electricity_from_battery)*Data['Price'] * 1000,
    'Return Exports' : PV_Excess * Data['Price'] * 1000,
    'PE Grid' : Grid_Electricity*Grid_PEF,
    'PE Solar' : (PV_Consumption + Electricity_from_battery) * Solar_PEF,
    'PE Exports' : PV_Excess * Solar_PEF
})
Optimization_data['PE Final Balance'] = Optimization_data['PE Grid'] 
        + Optimization_data['PE Solar'] - Optimization_data['PE Exports']
    
##########################################################################################
##########################################################################################
##########################################################################################
# Export
months = Optimization_data['Month'].unique()
Optimization_monthly = Optimization_data.groupby(['Month']).sum().reindex(months).drop('Hour', axis = 1)

Optimization_data.to_csv('Optimization_hourly.csv')
Optimization_monthly.to_csv('Optimization_monthly.csv')